# Zika Map 

In [44]:
import pandas as pd
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import wget

In [2]:
py.sign_in('hannah101101', '9qyvhEf5VpOG5cdufpLb')

## Example from `plotly`

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
df.head()


,code,state,category,total exports,beef,pork,poultry,dairy,fruits fresh,fruits proc,total fruits,veggies fresh,veggies proc,total veggies,corn,wheat,cotton
0,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
1,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
2,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
3,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
4,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95


In [4]:
for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['state'] + '<br>' +\
    'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
    'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
    'Wheat '+df['wheat']+' Corn '+df['corn']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['total exports'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

## [ `plotly` ebola example](https://plot.ly/python/choropleth-maps/)

### Read in the data
First, we will read in the Ebola case count data. 

There are six columns in this dataframe: 
* **Country** 
* **Month**
* **Year** 
* **Lat** 
* **Lon** 
* **Value** (number of ebola cases)

### Set-up the plot

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_ebola.csv')
df.head()

,Country,Month,Year,Lat,Lon,Value
0,Guinea,3,14,9.95,-9.7,122.0
1,Guinea,4,14,9.95,-9.7,224.0
2,Guinea,5,14,9.95,-9.7,291.0
3,Guinea,6,14,9.95,-9.7,413.0
4,Guinea,7,14,9.95,-9.7,460.0


#### Clean/setup the cases

The final plot shows the **number of cases** in several **countries** over a four month period. 
The first thing we need to do is create for each **month** a **dictionary** that holds the **data** for that **month** as well as the **formatting** 

We will store all of these **dictionaries** in a **list** called `cases`. 

In [6]:
# Which colors do we want to use? 
# use https://www.w3schools.com/colors/colors_rgb.asp to look at the RGB codes
colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']

cases = []  # this will hold all of the month-specific dictionaries. 
months = {6:'June',7:'July',8:'Aug',9:'Sept'}  # this dictionary translate numerical month codes into names


# for each month we need to 
# (1) retrieve the lat/long coordinates from the dataframe
# (2) give the month text and a name 
# determine how large the marker should be

for i in range(6,10)[::-1]:  # loop through the months in reverse order 
    print(f"We are processing {months[i]} ({i}):")
    cases.append(go.Scattergeo(
        lon = df[ df['Month'] == i ]['Lon'], 
        lat = df[ df['Month'] == i ]['Lat'],
        text = df[ df['Month'] == i ]['Value'],
        name = months[i],
        marker = dict(
            size = df[ df['Month'] == i ]['Value']/50,
            color = colors[i-6],
            line = dict(width = 0)
        ),
    ) )
print("We are done processing the months.")


# We want the name of the country to show beneath each circle. 
# We will edit the first month (Sept) to add the Country name
cases[0]['text'] = df[ df['Month'] == 9 ]['Value'].map('{:.0f}'.format).astype(str)+' '+\
    df[ df['Month'] == 9 ]['Country']
cases[0]['mode'] = 'markers+text'
cases[0]['textposition'] = 'bottom center'

We are processing Sept (9):
We are processing Aug (8):
We are processing July (7):
We are processing June (6):
We are done processing the months.


#### clean/setup the insets

We want to show two plots: 
* A map of Africa with our countries of interest highlighted 
* A zoom-in on West Africa with our case count information 

We are going to store the information in a **list** called `inset`. 
The first item in the **list** will be a `go.Choropleth` object which holds the information on how to plot the zoom-in map. 
The second item in the **list** will a `go.Scattergeo` object which holds the information on how to the plot map of Africa. 

In [7]:
inset = [
    go.Choropleth(
        locationmode = 'country names',
        locations = df[ df['Month'] == 9 ]['Country'],
        z = df[ df['Month'] == 9 ]['Value'],  # the color/size should be case counts
        text = df[ df['Month'] == 9 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo2'
    ),
    go.Scattergeo(
        lon = [21.0936],
        lat = [7.1881],
        text = ['Africa'],
        mode = 'text',  # When you hover, show information held in `cases` text
        showlegend = False,
        geo = 'geo2'
    )
]

#### clean/setup the layout

In [8]:
layout = go.Layout(
    title = 'Ebola cases reported by month in West Africa 2014<br> \
Source: <a href="https://data.hdx.rwlabs.org/dataset/rowca-ebola-cases">\
HDX</a>',
    geo = dict(
        resolution = 50,
        scope = 'africa',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'Mercator'
        ),
        lonaxis = dict( range= [ -15.0, -5.0 ] ),
        lataxis = dict( range= [ 0.0, 12.0 ] ),
        domain = dict(
            x = [ 0, 1 ],
            y = [ 0, 1 ]
        )
    ),
    geo2 = dict(
        scope = 'africa',
        showframe = False,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        showcountries = False,
        domain = dict(
            x = [ 0, 0.6 ],
            y = [ 0, 0.6 ]
        ),
        bgcolor = 'rgba(255, 255, 255, 0.0)',
    ),
    legend = dict(
           traceorder = 'reversed'
    )
)

### Plot the final figure

In [9]:
fig = go.Figure(layout=layout, data=cases+inset)  # give plotly the layout we want and the data
py.iplot(fig, validate=False, filename='West Africa Ebola cases 2014')  # print it out 

## Zika map

We want to make a similar map for Zika in South America. 

### Read in the Data

In [72]:
py.sign_in('hannah101101', '9qyvhEf5VpOG5cdufpLb')

url = "https://raw.githubusercontent.com/hutch-gwc/science_fair/master/data/infectious_diseases/zika_southAmerica.csv" # insert your text file URL here
filename = wget.download(url)

df = pd.read_csv(filename)
df.head()


,susp/con ZIKV cases,Unnamed: 1,Unnamed: 2,Argentina,Bolivia,Brazil,Colombia,Ecuador,French Guiana,Guyana,Paraguay,Peru,Suriname,Venezuela
0,year,date,Epi Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,3-Jan,1,0.0,2.0,7619.0,2883.0,1.0,41.0,0.0,22.0,0.0,245.0,241.0
2,NaN,10-Jan,2,0.0,0.0,7785.0,3374.0,5.0,75.0,0.0,37.0,0.0,228.0,535.0
3,NaN,17-Jan,3,0.0,3.0,8861.0,4558.0,9.0,106.0,0.0,27.0,0.0,394.0,1203.0
4,NaN,24-Jan,4,0.0,0.0,9772.0,5804.0,4.0,186.0,0.0,31.0,0.0,408.0,2713.0


#### Clean col names

As you can see, the column names did not come through exactly as we would want them to. 
We are going to rename the first three columns and then drop the first row. 

In [73]:
df = df.rename(index=str, columns={"susp/con ZIKV cases": "year", "Unnamed: 1": "date", "Unnamed: 2": "Epi Week"})
df = df.iloc[1:]
df.head()

,year,date,Epi Week,Argentina,Bolivia,Brazil,Colombia,Ecuador,French Guiana,Guyana,Paraguay,Peru,Suriname,Venezuela
1,2016,3-Jan,1,0.0,2.0,7619.0,2883.0,1.0,41.0,0.0,22.0,0.0,245.0,241.0
2,NaN,10-Jan,2,0.0,0.0,7785.0,3374.0,5.0,75.0,0.0,37.0,0.0,228.0,535.0
3,NaN,17-Jan,3,0.0,3.0,8861.0,4558.0,9.0,106.0,0.0,27.0,0.0,394.0,1203.0
4,NaN,24-Jan,4,0.0,0.0,9772.0,5804.0,4.0,186.0,0.0,31.0,0.0,408.0,2713.0
5,NaN,31-Jan,5,0.0,2.0,12091.0,6358.0,2.0,199.0,3.0,46.0,0.0,352.0,5011.0


#### Clean year col
Next, we need to fill in the missing values for the year column. 
The first 52 rows should all say 2016 and the next 52 rows should all say 2017. 

In [74]:
df.loc[0:52,'year'] = 2016
df.loc[52:,'year'] = 2017
df.head()

,year,date,Epi Week,Argentina,Bolivia,Brazil,Colombia,Ecuador,French Guiana,Guyana,Paraguay,Peru,Suriname,Venezuela
1,2016,3-Jan,1,0.0,2.0,7619.0,2883.0,1.0,41.0,0.0,22.0,0.0,245.0,241.0
2,2016,10-Jan,2,0.0,0.0,7785.0,3374.0,5.0,75.0,0.0,37.0,0.0,228.0,535.0
3,2016,17-Jan,3,0.0,3.0,8861.0,4558.0,9.0,106.0,0.0,27.0,0.0,394.0,1203.0
4,2016,24-Jan,4,0.0,0.0,9772.0,5804.0,4.0,186.0,0.0,31.0,0.0,408.0,2713.0
5,2016,31-Jan,5,0.0,2.0,12091.0,6358.0,2.0,199.0,3.0,46.0,0.0,352.0,5011.0


#### Melt dataframe

We want the there to be a column called `country` and a column called `cases` rather than each country being its own column. 

In [75]:
df = pd.melt(df, id_vars=["year", "date", "Epi Week"], var_name='Country', value_name='Cases')
df.head()

,year,date,Epi Week,Country,Cases
0,2016,3-Jan,1,Argentina,0.0
1,2016,10-Jan,2,Argentina,0.0
2,2016,17-Jan,3,Argentina,0.0
3,2016,24-Jan,4,Argentina,0.0
4,2016,31-Jan,5,Argentina,0.0


#### Extract month from the data 
We want to add a new column which is the month for each of the cases. 
We can do this by pulling out the month from the data. 
Each date is in the format `D-MON` so we can "split" the date on the character `-` and only take the information from the right hand side. 

In [76]:
df["month"] = df["date"].apply(lambda x: x.split("-")[1])
df

,year,date,Epi Week,Country,Cases,month
0,2016,3-Jan,1,Argentina,0.0,Jan
1,2016,10-Jan,2,Argentina,0.0,Jan
2,2016,17-Jan,3,Argentina,0.0,Jan
3,2016,24-Jan,4,Argentina,0.0,Jan
4,2016,31-Jan,5,Argentina,0.0,Jan
5,2016,7-Feb,6,Argentina,0.0,Feb
6,2016,14-Feb,7,Argentina,1.0,Feb
7,2016,21-Feb,8,Argentina,0.0,Feb
8,2016,28-Feb,9,Argentina,0.0,Feb
9,2016,6-Mar,10,Argentina,0.0,Mar


#### Sum cases by country/year 
"we are trying to add up the cases for each month for each country. It can count Argentina, but then gets tired and thinks Bolivia is 5. Problems" 

In [77]:
cases_by_year = []
for name, group in df.groupby(["Country", "year"]):
    cases = group["Cases"].sum()
    cases_by_year.append((name[0], name[1], cases))
cases_by_year = pd.DataFrame(cases_by_year, columns=["Country", "Year", "Cases"])
cases_by_year.head()

,Country,Year,Cases
0,Argentina,2016,26.0
1,Argentina,2017,825.0
2,Bolivia,2016,343.0
3,Bolivia,2017,1353.0
4,Brazil,2016,237234.0


#### Add in Lat/Long

In [78]:
lat_long = pd.read_csv("https://raw.githubusercontent.com/hutch-gwc/science_fair/master/booths/infectious_diseases/lat_long.csv")
lat_long.head()

,Country,lat,long
0,Argentina,-38.4,-63.6
1,Bolivia,-16.3,-63.6
2,Brazil,-14.2,-51.9
3,Colombia,4.6,-74.3
4,Ecuador,-1.8,-78.2


In [79]:
cases_by_year = pd.merge(cases_by_year, lat_long, on="Country")
cases_by_year.head()

,Country,Year,Cases,lat,long
0,Argentina,2016,26.0,-38.4,-63.6
1,Argentina,2017,825.0,-38.4,-63.6
2,Bolivia,2016,343.0,-16.3,-63.6
3,Bolivia,2017,1353.0,-16.3,-63.6
4,Brazil,2016,237234.0,-14.2,-51.9


### Set-up plot

#### Cases

In [18]:
# cases_by_year = cases_by_year[~cases_by_year['Country'].isin(["Brazil", "Colombia", "French Guiana"])]
cases_by_year

,Country,Year,Cases,lat,long
0,Argentina,2016,26.0,-38.4,-63.6
1,Argentina,2017,825.0,-38.4,-63.6
2,Bolivia,2016,343.0,-16.3,-63.6
3,Bolivia,2017,1353.0,-16.3,-63.6
8,Ecuador,2016,1884.0,-1.8,-78.2
9,Ecuador,2017,1957.0,-1.8,-78.2
12,Guyana,2016,33.0,4.9,-58.9
13,Guyana,2017,0.0,4.9,-58.9
14,Paraguay,2016,594.0,-23.4,-58.4
15,Paraguay,2017,95.0,-23.4,-58.4


In [34]:
colors = {2016: 'rgb(189,215,231)', 2017: 'rgb(238, 105, 234)'}
cases = []  # this will hold all of the year-specific dictionaries. 

for year, group in cases_by_year.groupby(["Year"]):
    print(f"We are processing year {year}.")
    cases.append(go.Scattergeo(
        lon = group['long'], 
        lat = group['lat'],
        text = group['Cases'],
        name = year,
        marker = dict(
            size = group['Cases']/50,
            color = colors[year],
            line = dict(width = 0)
        ),
    ) )
print("We are done processing the years.")


# We want the name of the country to show beneath each circle. 
# We will edit the first month (Sept) to add the Country name
cases[0]['text'] = cases_by_year[ cases_by_year['Year'] == 2017 ]['Cases'].map('{:.0f}'.format).astype(str)+' '+\
    cases_by_year[ cases_by_year['Year'] == 2017 ]['Country']
cases[0]['mode'] = 'markers+text'
cases[0]['textposition'] = 'bottom center'

We are processing year 2016.
We are processing year 2017.
We are done processing the years.


#### inset

In [20]:
inset = [
    go.Choropleth(
        locationmode = 'country names',
        locations = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        z = cases_by_year[ cases_by_year['Year'] == 2017 ]['Cases'],  # the color/size should be case counts
        text = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo2'
    ),
    go.Scattergeo(
        lon = [-14],
        lat = [-52],
        text = ['South America'],
        mode = 'text',  # When you hover, show information held in `cases` text
        showlegend = False,
        geo = 'geo2'
    )
]

#### layout

In [21]:
layout = go.Layout(
    title = 'Zika Cases',
    geo = dict(
        resolution = 50,
        scope = 'south america',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'Mercator'
        ),
        lonaxis = dict( range= [ -90.0, 50 ] ),
        lataxis = dict( range= [ -60, 15 ] ),
        domain = dict(
            x = [ 0, 1 ],
            y = [ 0, 1 ]
        )
    ),
    geo2 = dict(
        scope = 'south america',
        showframe = False,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        showcountries = False,
        domain = dict(
            x = [ 0.2, 0.8 ],
            y = [ 0.0, 0.5 ]
        ),
        bgcolor = 'rgba(255, 255, 255, 0.0)',
    ),
    legend = dict(
           traceorder = 'reversed'
    )
)

### Plot!

In [22]:
fig = go.Figure(layout=layout, data=cases+inset)  # give plotly the layout we want and the data
py.iplot(fig, validate=False, filename='West Africa Ebola cases 2014')  # print it out 

## map take two

In [40]:
for year in cases_by_year["Year"].unique():
    data = [ dict(
            type='choropleth',
            colorscale = scl,
            autocolorscale = False,
            locations = cases_by_year[ cases_by_year['Year'] == year ]['Country'],
            z = cases_by_year[ cases_by_year['Year'] == year ]['Cases'],
            locationmode = 'country names',
    #         text = df['text'],
            marker = dict(
                line = dict (
                    color = 'rgb(255,255,255)',
                    width = 2
                ) ),
            colorbar = dict(
                title = "Millions USD")
            ) ]

    layout = dict(
            title = f'Zika in {year}',
            geo = dict(
                scope='south america',
    #             projection=dict( type='albers usa' ),
                showlakes = True,
                lakecolor = 'rgb(255, 255, 255)'),
                 )

    fig = dict( data=data, layout=layout )
    py.iplot( fig, filename='d3-cloropleth-map' )

In [49]:
trace1 = go.Choropleth(
        locationmode = 'country names',
        locations = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        z = cases_by_year[ cases_by_year['Year'] == 2017 ]['Cases'],  # the color/size should be case counts
        text = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo2'
    )
trace2 = go.Choropleth(
        locationmode = 'country names',
        locations = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        z = cases_by_year[ cases_by_year['Year'] == 2017 ]['Cases'],  # the color/size should be case counts
        text = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo2'
    )


fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(trace1, 1,1)
fig.append_trace(trace2)

# fig['layout'].update(height=600, width=800, title='i <3 annotations and subplots')
# py.iplot(fig, filename='simple-subplot-with-annotations')
# This is the format of your plot grid:
# [ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



PlotlyDictKeyError: 'xaxis' is not allowed in 'choropleth'

Path To Error: ['xaxis']

Valid attributes for 'choropleth' at path [] under parents []:

    ['autocolorscale', 'colorbar', 'colorscale', 'customdata',
    'customdatasrc', 'geo', 'hoverinfo', 'hoverinfosrc', 'hoverlabel',
    'ids', 'idssrc', 'legendgroup', 'locationmode', 'locations',
    'locationssrc', 'marker', 'name', 'opacity', 'reversescale',
    'selected', 'selectedpoints', 'showlegend', 'showscale', 'stream',
    'text', 'textsrc', 'type', 'uid', 'unselected', 'visible', 'z',
    'zauto', 'zmax', 'zmin', 'zsrc']

Run `<choropleth-object>.help('attribute')` on any of the above.
'<choropleth-object>' is the object at []

## map take 3

In [80]:
df = cases_by_year.copy()
df.head()

,Country,Year,Cases,lat,long
0,Argentina,2016,26.0,-38.4,-63.6
1,Argentina,2017,825.0,-38.4,-63.6
2,Bolivia,2016,343.0,-16.3,-63.6
3,Bolivia,2017,1353.0,-16.3,-63.6
4,Brazil,2016,237234.0,-14.2,-51.9


In [161]:
df = cases_by_year.copy()

data = []
layout = dict(
            title = 'Zika in South America',
            geo = dict(
                scope='south america',
    #             projection=dict( type='albers usa' ),
                showlakes = True,
                lakecolor = 'rgb(255, 255, 255)'),
                 )
years = df['Year'].unique()

for i, year in enumerate(years):
    print(year)
    geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
    print(geo_key)
    lons = list(df[ df['Year'] == year ]['long'])
    lats = list(df[ df['Year'] == year ]['lat'])
    data.append(
    dict(
            type='choropleth',
            colorscale = scl,
            autocolorscale = False,
            locations = df[ df['Year'] == year ]['Country'],
            z = df[ df['Year'] == year ]['Cases'],
            locationmode = 'country names',
    #         text = df['text'],
            marker = dict(
                line = dict (
                    color = 'rgb(255,255,255)',
                    width = 2
                ) ),
            colorbar = dict(
                title = "Cases")
            )
    )
    # Year markers
    data.append(
        dict(
            type = 'scattergeo',
            showlegend = False,
            lon = [-63],
            lat = [16],
            geo = geo_key,
            text = [year],
            mode = 'text',
        )
    )
    layout[geo_key] = dict(
        scope = 'south america',
        showland = True,
        landcolor = 'rgb(229, 229, 229)',
        showcountries = True,
        domain = dict( x = [], y = [] ),
        subunitcolor = "rgb(255, 255, 255)",
    )

z = 0
COLS = 2
ROWS = 1
for y in reversed(range(ROWS)):
    for x in range(COLS):
        geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
        layout[geo_key]['domain']['x'] = [float(x)/float(COLS), float(x+1)/float(COLS)]
        layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), float(y+1)/float(ROWS)]
        z=z+1
#         if z > 42:
#             break
print(layout)
fig = { 'data':data, 'layout':layout }
py.iplot( fig, filename='Zika in South America', height=900, width=1000 )

2016
geo
2017
geo2
{'title': 'Zika in South America', 'geo': {'scope': 'south america', 'showland': True, 'landcolor': 'rgb(229, 229, 229)', 'showcountries': True, 'domain': {'x': [0.0, 0.5], 'y': [0.0, 1.0]}, 'subunitcolor': 'rgb(255, 255, 255)'}, 'geo2': {'scope': 'south america', 'showland': True, 'landcolor': 'rgb(229, 229, 229)', 'showcountries': True, 'domain': {'x': [0.5, 1.0], 'y': [0.0, 1.0]}, 'subunitcolor': 'rgb(255, 255, 255)'}}


In [134]:
# for year in cases_by_year["Year"].unique():

## first 2016 
year = 2016
layout = dict(
            title = 'Zika in South America',
            geo = dict(
                scope='south america',
    #             projection=dict( type='albers usa' ),
                showlakes = True,
                lakecolor = 'rgb(255, 255, 255)'),
                 )
data_2016 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = cases_by_year[ cases_by_year['Year'] == year ]['Country'],
        z = cases_by_year[ cases_by_year['Year'] == year ]['Cases'],
        locationmode = 'country names',
#         text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout["2016"] = dict(
    scope = 'south america',
    showland = True,
    landcolor = 'rgb(229, 229, 229)',
    showcountries = True,
    domain = dict( x = [0, 0], y = [0, 0] ),
    subunitcolor = "rgb(255, 255, 255)",
)


fig_2016 = dict( data=data_2016, layout=layout_2016 )

## now 2017
year = 2017
layout = dict(
            title = 'Zika in South America',
            geo = dict(
                scope='south america',
    #             projection=dict( type='albers usa' ),
                showlakes = True,
                lakecolor = 'rgb(255, 255, 255)'),
                 )
data_2017 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = cases_by_year[ cases_by_year['Year'] == year ]['Country'],
        z = cases_by_year[ cases_by_year['Year'] == year ]['Cases'],
        locationmode = 'country names',
#         text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout["2017"] = dict(
    scope = 'south america',
    showland = True,
    landcolor = 'rgb(229, 229, 229)',
    showcountries = True,
    domain = dict( x = [0, 0], y = [0, 0] ),
    subunitcolor = "rgb(255, 255, 255)",
)

fig_2017 = dict( data=data_2017, layout=layout_2017 )
fig_2016 = dict( data=data_2016, layout=layout )
py.iplot( fig_2016, filename='d3-cloropleth-map' )

PlotlyError: Invalid 'figure_or_data' argument. Plotly will not be able to properly parse the resulting JSON. If you want to send this 'figure_or_data' to Plotly anyway (not recommended), you can set 'validate=False' as a plot option.
Here's why you're seeing this error:

'2017' is not allowed in 'layout'

Path To Error: ['layout']['2017']

Valid attributes for 'layout' at path ['layout'] under parents ['figure']:

    ['angularaxis', 'annotations', 'autosize', 'bargap', 'bargroupgap',
    'barmode', 'barnorm', 'boxgap', 'boxgroupgap', 'boxmode', 'calendar',
    'colorway', 'datarevision', 'direction', 'dragmode', 'font', 'geo',
    'height', 'hiddenlabels', 'hiddenlabelssrc', 'hidesources',
    'hoverdistance', 'hoverlabel', 'hovermode', 'images', 'legend',
    'mapbox', 'margin', 'orientation', 'paper_bgcolor', 'plot_bgcolor',
    'polar', 'radialaxis', 'scene', 'separators', 'shapes', 'showlegend',
    'sliders', 'spikedistance', 'ternary', 'title', 'titlefont',
    'updatemenus', '

## take 4

In [169]:
inset = [
    go.Choropleth(
        locationmode = 'country names',
        locations = cases_by_year[ cases_by_year['Year'] == 2016 ]['Country'],
        z = cases_by_year[ cases_by_year['Year'] == 2016 ]['Cases'],  # the color/size should be case counts
        text = cases_by_year[ cases_by_year['Year'] == 2016 ]['Country'],
        autocolorscale = False,
        showscale = False,
        geo = 'geo2'
    ),
    go.Choropleth(
        locationmode = 'country names',
        locations = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        z = cases_by_year[ cases_by_year['Year'] == 2017 ]['Cases'],  # the color/size should be case counts
        text = cases_by_year[ cases_by_year['Year'] == 2017 ]['Country'],
        autocolorscale = False,
        showscale = False,
        geo = 'geo1'
    )
]

layout2 = go.Layout(
    title = 'Zika Cases',
    geo1 = dict(
        resolution = 50,
        scope = 'south america',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'Mercator'
        ),
        lonaxis = dict( range= [ -90.0, 50 ] ),
        lataxis = dict( range= [ -60, 15 ] ),
        domain = dict(
            x = [ 0.0, 0.5],
            y = [ 0,1]
        )
    ),
    geo = dict(
        resolution = 50,
        scope = 'south america',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'Mercator'
        ),
        lonaxis = dict( range= [ -90.0, 50 ] ),
        lataxis = dict( range= [ -60, 15 ] ),
        domain = dict(
            x = [0.5, 1.0],
            y = [ 0, 1 ]
        )
)
)

In [171]:
fig = go.Figure(layout=layout2[0], data=inset[0])  # give plotly the layout we want and the data
py.iplot(fig, validate=False, filename='West Africa Ebola cases 2014')  # print it out 

TypeError: expected string or bytes-like object